In [1]:
import Pkg
Pkg.activate("..")

using SCPToolbox
using ECOS

  Activating project at `~/Documents/SCPToolbox_tutorial`
┌ Info: Precompiling SCPToolbox [bd2bc758-a5dc-11eb-050f-6b2434548817]
└ @ Base loading.jl:1423


# Conic Linear Programs

General form:

\begin{align}
\underset{x\in\mathbb{R}^n}{{\text{minimize}}} &~ c^\top x \\
\text{subject to} &~Ax+b \in \mathbb{K}
\end{align}

where $c\in\mathbb{R}^n,~b\in\mathbb{R}^m,~A\in\mathbb{R}^{m\times n}$ are problem parameters, and $\mathbb{K}$ is a cartesian product of convex cones. The cones supported by `SCPToolbox.jl` are described in the table below.

|    Label        | Notation     |     Name      | Definition |
|:---------------:|:----:|:-------------:|:----------:|
|`UNCONSTRAINED`  |  $\mathbb{K}_{ \lesseqgtr}$ | Unconstrained cone  |  $$\{x\in\mathbb{R}^n\}$$ |
|`ZERO`           |  $\mathbb{K}_{=0}$ | Zero cone             | $\{x\in\mathbb{R}^n~|~x=0\}$                               |
|`NONPOS`         |  $\mathbb{K}_{\leq 0}$ | Non-positive orthant  | $\{x\in\mathbb{R}^n~|~x\leq 0\}$                           |
|`L1`             |  $\mathbb{K}_{1}$ | $l_1$ norm cone       | $\{(t,x)\in\mathbb{R}^n\times\mathbb{R}~|~\|x\|_1\leq t\}$ |
|`SOC`             |  $\mathbb{K}_{2}$ | $l_2$ norm cone       | $\{(t,x)\in\mathbb{R}^n\times\mathbb{R}~|~\|x\|_2\leq t\}$ |
| `LINF` |  $\mathbb{K}_{\infty}$ |  $l_\infty$ norm cone |  $\{(t,x)\in\mathbb{R}^n\times\mathbb{R}~|~\|x\|_\infty\leq t\}$  |
| `GEOM` |  $\mathbb{K}_{\text{GM}}$ | Geometric mean cone | $ \left\{ (t,x)\in\mathbb{R}^n\times\mathbb{R}~\middle|~\left(\prod_{i=1}^n x_i\right)^{1/n} \geq t \right\}$ |
| `EXP` |  $\mathbb{K}_{\text{EXP}}$ | Exponential cone | $\{(x,y,z)\in \mathbb{R}\times \mathbb{R} \times \mathbb{R} ~|~ y\,\text{exp}(x/y) \leq z,~y>0\}$ |

The optimization problem shown above can be solved using state-of-the-art solvers like `ECOS`, `Gurobi` and `Mosek`. However, they typically need a parser to convert the problem data to a canonical form. Hence such solvers are not user-friendly to interface with and solve problems which are not natively expressed in the canonical forms. 

In `SCPToolbox.jl`, henceforth refered to as the toolbox, a conic linear program is formulated by instantiating a `ConicProblem` object with the choice of solver and its associated options.  

In [2]:
opts = Dict("verbose" => 0)
pbm = ConicProgram(solver = ECOS.Optimizer, solver_options = opts)

Conic linear program

  Feasibility problem
  0 variables (0 blocks)
  0 parameters (0 blocks)
  0 constraints

  Variable argument
    0 elements
    0 blocks


  Parameter argument
    0 elements
    0 blocks


## Constraints

A conic constraint $y\in\mathbb{K}$ (say for $\mathbb{K} = \mathbb{K}_2$) can be modelled in the toolbox as follows:

In [18]:
params = Dict("n" => 4, "a" => 100)
opts = Dict("verbose" => 0)
pbm = ConicProgram(params; solver = ECOS.Optimizer, solver_options = opts)

x = @new_variable(pbm, length(params["n"]),"x")
t = @new_variable(pbm,"t")

# Define the constraint
cstr = @add_constraint(
    pbm, SOC, "my-soc", (x, t),
    begin
        local x, t = arg
        vcat(pars["a"]*t, x)
    end)

Name: my-soc
Cone f(x,p)∈K, where:
K is a second-order cone, {(t, x)∈ℝ×ℝⁿ : ‖x‖₂≤t}
f(x,p) = 
  100 t
  x
Affine function
Arguments:
  x (block 1) : 1
  t (block 2) : 2


In [19]:
variables(pbm,"^x\$") == x

true

Note the use of [regular expressions](https://en.wikipedia.org/wiki/Regular_expression) for searching though the list of variables names.

In [45]:
constraints(pbm, "^my-soc")

0 constraints

## Examples

Many simple constraint set typical in a trajectory optimization problem like box, ball, halfspace and hyperplane can be represented as conic constraints.

### Box

A box with bounds $l,u\in\mathbb{R}$ with $l<u$ can be formulated as as non-positive orthant cone:

$$
\{x\in\mathbb{R}^n~|~l\leq x \leq u\}
$$

$$
l\leq x \leq u \iff \left[ \begin{array}{c}I\\ -I\end{array}\right]x + \left[\begin{array}{c}-u\mathbf{1}\\ l\mathbf{1} \end{array}\right] \in \mathbb{K}_{\leq 0}
$$

In [27]:
params = Dict("n" => 2, "l" => 2, "u" => 9)
A = [1  0;0 1;-1 0;0 -1]
b = [-params["u"];-params["u"];params["l"];params["l"]]

pbm = ConicProgram(params)

x = @new_variable(pbm, params["n"],"x")

# Define the constraint
cstr = @add_constraint(
    pbm, NONPOS, "my-box", (x,),
    begin
        local x, = arg
        A*x+b
    end)

Name: my-box
Cone f(x,p)∈K, where:
K is a nonpositive orthant cone, {z : z≤0}
f(x,p) = 
  x[1] - 9
  x[2] - 9
  -x[1] + 2
  -x[2] + 2
Affine function
Arguments:
  x (block 1) : 1:2


### Ball

A ball with radius $r \in \mathbb{R}_+$ set can be represented as a $l_2$ norm cone (second-order cone):

$$
\{x\in\mathbb{R}^n~|~\|x\|_2\leq r\}
$$

$$
\|x\|_2\leq r \iff \left[ \begin{array}{c} 0 \\ I \end{array} \right] x + \left[ \begin{array}{c} r \\ 0 \end{array} \right]\in \mathbb{K}_2
$$

In [30]:
params = Dict("n" => 2, "r" => 4)
A = [0  0;1 0;0 1]
b = [params["r"];0;0]

pbm = ConicProgram(params)

x = @new_variable(pbm, params["n"],"x")

# Define the constraint
cstr = @add_constraint(
    pbm, SOC, "my-ball", (x,),
    begin
        local x, = arg
        A*x+b
    end)

Name: my-ball
Cone f(x,p)∈K, where:
K is a second-order cone, {(t, x)∈ℝ×ℝⁿ : ‖x‖₂≤t}
f(x,p) = 
  4
  x[1]
  x[2]
Affine function
Arguments:
  x (block 1) : 1:2


### Hyperplane

A hyperplane defined by $a\in \mathbb{R}^n$ and $b\in\mathbb{R}$ can be represented as a zero cone:

$$
\left\{x\in\mathbb{R}^n~|~a^\top x = b\right\}
$$

$$
a^\top x = b \iff \left[a^\top\right]x + (-b) \in \mathbb{K}_{=0}
$$

In [44]:
params = Dict("n" => 2, "a" => [1.1;3.2], "b" => 9)
A = Matrix(transpose(params["a"]))
b = -params["b"]

@show A

pbm = ConicProgram(params)

x = @new_variable(pbm, params["n"],"x")

# Define the constraint
cstr = @add_constraint(
    pbm, ZERO, "my-hypln", (x,),
    begin
        local x, = arg
        A*x .+ b
    end)

A = [1.1 3.2]


Name: my-hypln
Cone f(x,p)∈K, where:
K is a zero cone, {z : z=0}
f(x,p) = 
  1.1 x[1] + 3.2 x[2] - 9
Affine function
Arguments:
  x (block 1) : 1:2


### Halfspace

A halfspace defined by $a\in \mathbb{R}^n$ and $b\in\mathbb{R}$ can be represented as a non-positive orthant cone:

$$
\left\{x\in\mathbb{R}^n~|~a^\top x \leq b\right\}
$$

$$
a^\top x \leq b \iff \left[a^\top\right] x + (-b) \in \mathbb{K}_{\leq 0}
$$

In [43]:
params = Dict("n" => 2, "a" => [1.1;3.2], "b" => 9)
A = Matrix(transpose(params["a"]))
b = -params["b"]

@show A*[1;2]

pbm = ConicProgram(params)

x = @new_variable(pbm, params["n"],"x")

# Define the constraint
cstr = @add_constraint(
    pbm, NONPOS, "my-hlfspace", (x,),
    begin
        local x, = arg
        println(size(A*x))
        A*x .+ b
    end)

A * [1; 2] = [7.5]
(1,)


Name: my-hlfspace
Cone f(x,p)∈K, where:
K is a nonpositive orthant cone, {z : z≤0}
f(x,p) = 
  1.1 x[1] + 3.2 x[2] - 9
Affine function
Arguments:
  x (block 1) : 1:2
